In [1]:
import numpy as np
import pandas as pd
import requests
from bs4 import BeautifulSoup
import json

In [2]:
r = requests.get('https://httpbin.org/user-agent')
useragent = json.loads(r.text)['user-agent']
headers = {'User-Agent': useragent,
           'From': 'urn8he@virginia.edu'}
print(useragent)

python-requests/2.32.2


In [3]:
url = 'https://www.rottentomatoes.com/browse/movies_in_theaters/sort:a_z?page=5'
r = requests.get(url, headers = headers)
r

<Response [200]>

In [4]:
mysoup = BeautifulSoup(r.text, 'html.parser')

# Goal 1



Get a list of hyperlinks to individual movies' pages

In [5]:
hreflist = mysoup.find_all('a', attrs = {'data-track' : 'scores'})

In [6]:
hreflist[3]['href']

'/m/bad_boys_ride_or_die'

In [7]:
movielinks =['https://www.rottentomatoes.com' + x['href'] for x in hreflist]

In [8]:
movielinks

['https://www.rottentomatoes.com/m/a_man_of_reason',
 'https://www.rottentomatoes.com/m/a_quiet_place_day_one',
 'https://www.rottentomatoes.com/m/babes',
 'https://www.rottentomatoes.com/m/bad_boys_ride_or_die',
 'https://www.rottentomatoes.com/m/banel_and_adama',
 'https://www.rottentomatoes.com/m/beverly_hills_cop_axel_f',
 'https://www.rottentomatoes.com/m/blue_lock_the_movie_episode_nagi',
 'https://www.rottentomatoes.com/m/boneyard_2024',
 'https://www.rottentomatoes.com/m/chestnut',
 'https://www.rottentomatoes.com/m/close_encounters_of_the_third_kind',
 'https://www.rottentomatoes.com/m/daddio_2023',
 'https://www.rottentomatoes.com/m/dandelion_2024',
 'https://www.rottentomatoes.com/m/dead_whisper',
 'https://www.rottentomatoes.com/m/despicable_me_4',
 'https://www.rottentomatoes.com/m/talju',
 'https://www.rottentomatoes.com/m/escape_2023',
 'https://www.rottentomatoes.com/m/evil_does_not_exist',
 'https://www.rottentomatoes.com/m/firebrand_2023',
 'https://www.rottentomatoes

# Goal 2

Extract data from one movie page

In [9]:
url = movielinks[4]

In [10]:
r = requests.get(url, headers = headers)
mysoup = BeautifulSoup(r.text)

In [11]:
moviedata = {}

In [12]:
moviedata['title'] = mysoup.find('title').text. replace(' | Rotten Tomatoes', '')

In [13]:
moviedata

{'title': 'Banel & Adama'}

# Goal 3

In [14]:
mysoup.find('meta', attrs = {'name':'description'})

<meta content="Banel and Adama are fiercely in love. The young couple lives in a remote village in northern Senegal. For them, nothing else exists except each other. But for the rest of their tight-knit village, duty dictates that Adama soon accept the role of chief. The two lovers have other plans... until something in the air changes. The rains do not come; the cattle begin to die; the men leave. A rare debut feature that premiered in competition at the Cannes Film Festival, and Senegal's official submission to the Academy Awards®, Ramata-Toulaye Sy's Banel &amp; Adama is a lush and lyrical fable that soars to the heights of longing and descends deep into the realm of myth, sending its protagonists' idyllic love on a collision course with their community's customs. Because in this world, there is no room for passion, let alone chaos." name="description">
<meta content="summary" name="twitter:card">
<meta content="https://resizing.flixster.com/m6xyJ1VTaD9AQV_obahVyggORtA=/206x305/v2/h

In [15]:
moviedata['critic_score'] = mysoup.find('rt-button', slot = "criticScore").text.strip()
moviedata['audience_score'] = mysoup.find('rt-button', slot = "audienceScore").text.strip()

AttributeError: 'NoneType' object has no attribute 'text'

In [ ]:
moviedata['description'] = mysoup.find('meta', attrs = {'name':'description'})['content']

In [ ]:
moviedata['critic_consensus'] = mysoup.find('div', 'consensus').p.text

"Will Smith and Martin Lawrence remain good company even when Bad Boys strains to up the ante, proving there's still life left in this high-octane franchise."

# Goal 4

In [ ]:
def moviescrape(url):
    r = requests.get(url, headers = headers)
    mysoup = BeautifulSoup(r.text, 'html.parser')
    
    moviedata = {}
    
    moviedata['title'] = mysoup.find('title').text. replace(' | Rotten Tomatoes', '')
    moviedata['description'] = mysoup.find('meta', attrs = {'name':'description'})['content']
    moviedata['critic_score'] = mysoup.find('rt-button', slot = "criticsScore").text.strip()
    moviedata['audience_score'] = mysoup.find('rt-button', slot = "audienceScore").text.strip()
    
    try:
        moviedata['critic_consensus'] = mysoup.find('div', 'consensus').p.text
    except:
        moviedata['critic_consensus'] = ''
    
    cat = mysoup.find_all('div', "category-wrap")
    cats = [x.text.strip().split('\n\n\n') for x in cat]
    for c in cats:
        moviedata[c[0]] = c[1].replace('\n', '')
        
    return moviedata
    
    cats = {x[0]:x[1] for x in cats}
    

SyntaxError: expected ':' (2386028803.py, line 12)

In [ ]:
rt_data = [moviescrape(x) for x in movielinks]

NameError: name 'moviescrape' is not defined

In [ ]:
rt_dataframe = pd.DataFrame.from_records(rt_data)

NameError: name 'rt_data' is not defined